[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rsinghlab/pyaging/blob/main/tutorials/tutorial_histonemarkchipseq.ipynb) [![Open In nbviewer](https://img.shields.io/badge/View%20in-nbviewer-orange)](https://nbviewer.jupyter.org/github/rsinghlab/pyaging/blob/main/tutorials/tutorial_histonemarkchipseq.ipynb)

# Bulk histone mark ChIP-Seq

This tutorial is a brief guide for the implementation of the seven histone-mark-specific clocks and the pan-histone-mark clock developed ourselves. Link to [preprint](https://www.biorxiv.org/content/10.1101/2023.08.21.554165v3).

We just need two packages for this tutorial.

In [1]:
import pandas as pd
import pyaging as pya

## Download and load example data

Let's download an example of H3K4me3 ChIP-Seq bigWig file from the ENCODE project.

In [2]:
pya.data.download_example_data('ENCFF386QWG')

|-----> 🏗️ Starting download_example_data function
|-----------> Data found in pyaging_data/ENCFF386QWG.bigWig
|-----> 🎉 Done! [0.4776s]


|-----------> in progress: 1.0002%

|-----------> in progress: 2.0005%

|-----------> in progress: 3.0007%

|-----------> in progress: 4.0010%

...

|-----------> in progress: 97.0232%

|-----------> in progress: 98.0235%

|-----------> in progress: 99.0237%

|-----------> in progress: 100.0000%


|-----> 🎉 Done! [133.6851s]


To exemplify that multiple bigWigs can be turned into a df object at once, let's just repeat the file path.

In [3]:
df = pya.pp.bigwig_to_df(['pyaging_data/ENCFF386QWG.bigWig', 'pyaging_data/ENCFF386QWG.bigWig'])

|-----> 🏗️ Starting bigwig_to_df function
|-----> ⚙️ Load Ensembl genome metadata started
|-----------> Data found in pyaging_data/Ensembl-105-EnsDb-for-Homo-sapiens-genes.csv
|-----> ✅ Load Ensembl genome metadata finished [1.0212s]
|-----> ⚙️ Processing bigWig files started
|-----------> Processing file: pyaging_data/ENCFF386QWG.bigWig
|-----------> in progress: 100.0000%
|-----------> Processing file: pyaging_data/ENCFF386QWG.bigWig
|-----------> in progress: 100.0000%
|-----> ✅ Processing bigWig files finished [21.9402s]
|-----> 🎉 Done! [47.5545s]


|-----------> Downloading data to pyaging_data/Ensembl-105-EnsDb-for-Homo-sapiens-genes.csv


|-----------> in progress: 1.0402%

|-----------> in progress: 2.0804%

|-----------> in progress: 3.1206%

...

|-----------> in progress: 98.0929%

|-----------> in progress: 99.0938%

|-----------> in progress: 100.0000%


|-----> ✅ Processing bigWig files finished [10.9185s]


|-----> 🎉 Done! [26.0601s]


In [4]:
df.index = ['sample1', 'sample2'] # just to avoid an annoying anndata warning that samples have same names

In [5]:
df.head()

,ENSG00000223972,ENSG00000227232,ENSG00000278267,ENSG00000243485,ENSG00000284332,ENSG00000237613,ENSG00000268020,ENSG00000240361,ENSG00000186092,ENSG00000238009,...,ENSG00000237801,ENSG00000237040,ENSG00000124333,ENSG00000228410,ENSG00000223484,ENSG00000124334,ENSG00000270726,ENSG00000185203,ENSG00000182484,ENSG00000227159
sample1,0.028616,0.030415,0.027783,0.028616,0.028616,0.028616,0.044171,0.036474,0.030784,0.03181,...,0.034435,0.006822,1.413119,0.029424,0.140005,0.049786,0.069296,0.332126,0.028596,0.028616
sample2,0.028616,0.030415,0.027783,0.028616,0.028616,0.028616,0.044171,0.036474,0.030784,0.03181,...,0.034435,0.006822,1.413119,0.029424,0.140005,0.049786,0.069296,0.332126,0.028596,0.028616


## Convert data to AnnData object

AnnData objects are highly flexible and are thus our preferred method of organizing data for age prediction.

In [6]:
adata = pya.preprocess.df_to_adata(df)

|-----> 🏗️ Starting df_to_adata function
|-----> ⚙️ Create anndata object started
|-----> ✅ Create anndata object finished [0.0625s]
|-----> ⚙️ Add metadata to anndata started
|-----------? No metadata provided. Leaving adata.obs empty
|-----> ⚠️ Add metadata to anndata finished [0.0008s]
|-----> ⚙️ Log data statistics started
|-----------> There are 2 observations
|-----------> There are 62241 features
|-----------> Total missing values: 0
|-----------> Percentage of missing values: 0.00%
|-----> ✅ Log data statistics finished [0.0104s]
|-----> ⚙️ Impute missing values started
|-----------> No missing values found. No imputation necessary
|-----> ✅ Impute missing values finished [0.0045s]
|-----> 🎉 Done! [0.0874s]


|-----> ✅ Create anndata object finished [0.0272s]


|-----> ⚙️ Add metadata to anndata started


|-----------? No metadata provided. Leaving adata.obs empty


|-----> ⚠️ Add metadata to anndata finished [0.0004s]


...

|-----> ✅ Log data statistics finished [0.0011s]


|-----> ⚙️ Impute missing values started


|-----------> No missing values found. No imputation necessary


|-----> ✅ Impute missing values finished [0.0014s]


|-----> 🎉 Done! [0.0320s]


Note that the original DataFrame is stored in `X_original` under layers. This is what the `adata` object looks like:

In [7]:
adata

AnnData object with n_obs × n_vars = 2 × 62241
    var: 'percent_na'
    layers: 'X_original'

## Predict age

We can either predict one clock at once or all at the same time. For convenience, let's simply input a few clocks of interest at once. The function is invariant to the capitalization of the clock name. 

In [8]:
pya.pred.predict_age(adata, ['CamilloH3K4me3', 'CamilloH3K9me3', 'CamilloPanHistone'])

|-----> 🏗️ Starting predict_age function
|-----> ⚙️ Set PyTorch device started
|-----------> Using device: cpu
|-----> ✅ Set PyTorch device finished [0.0105s]
|-----> 🕒 Processing clock: camilloh3k4me3
|-----------> ⚙️ Load clock started
|-----------------> Data found in pyaging_data/camilloh3k4me3.pt
|-----------> ✅ Load clock finished [0.6478s]
|-----------> ⚙️ Check features in adata started
|-----------------> All features are present in adata.var_names.
|-----------------> Added prepared input matrix to adata.obsm[X_camilloh3k4me3]
|-----------> ✅ Check features in adata finished [0.0992s]
|-----------> ⚙️ Predict ages with model started
|-----------------> There is no preprocessing necessary
|-----------------> There is no postprocessing necessary
|-----------------> in progress: 100.0000%
|-----------> ✅ Predict ages with model finished [0.0169s]
|-----------> ⚙️ Add predicted ages and clock metadata to adata started
|-----------> ✅ Add predicted ages and clock metadata to adata

|-----------> Using device: cpu


|-----> ✅ Set PyTorch device finished [0.0005s]


|-----> 🕒 Processing clock: camilloh3k4me3


|-----------> ⚙️ Load clock started


...

|-----------------> in progress: 100.0000%


|-----------> ✅ Predict ages with model finished [0.0077s]


|-----------> ⚙️ Add predicted ages and clock metadata to adata started


|-----------> ✅ Add predicted ages and clock metadata to adata finished [0.0006s]


|-----> 🎉 Done! [15.1410s]


In [9]:
adata.obs.head()

,camilloh3k4me3,camilloh3k9me3,camillopanhistone
sample1,53.998544,44.3229,54.021884
sample2,53.998544,44.3229,54.021884


Having so much information printed can be overwhelming, particularly when running several clocks at once. In such cases, just set verbose to False.

In [ ]:
pya.data.download_example_data('ENCFF386QWG', verbose=False)
df = pya.pp.bigwig_to_df(['pyaging_data/ENCFF386QWG.bigWig', 'pyaging_data/ENCFF386QWG.bigWig'], verbose=False)
df.index = ['sample1', 'sample2']
adata = pya.preprocess.df_to_adata(df, verbose=False)
pya.pred.predict_age(adata, ['CamilloH3K4me3', 'CamilloH3K9me3', 'CamilloPanHistone'], verbose=False)

In [ ]:
adata.obs.head()

After age prediction, the clocks are added to `adata.obs`. Moreover, the percent of missing values for each clock and other metadata are included in `adata.uns`.

In [ ]:
adata

## Get citation

The doi, citation, and some metadata are automatically added to the AnnData object under `adata.uns[CLOCKNAME_metadata]`.

In [ ]:
adata.uns['camilloh3k4me3_metadata']